# Renko Chart

In [1]:
import pandas_datareader.data as pdr
import pandas as pd
import numpy as np
import datetime

from stocktrends import Renko

/anaconda3/envs/fx/lib/python3.8/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# Load data
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(3650)
data = pdr.get_data_yahoo('MSFT', start_date, end_date)
df = data.copy(deep=True)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-04-26,31.280001,30.860001,31.000000,31.110001,63649300.0,24.534153
2010-04-27,31.250000,30.750000,30.950001,30.850000,68730900.0,24.329115
2010-04-28,31.000000,30.620001,30.920000,30.910000,64557900.0,24.376423
2010-04-29,31.430000,30.670000,30.930000,31.000000,52665200.0,24.447399
2010-04-30,31.080000,30.520000,31.070000,30.540001,63214800.0,24.084637


In [3]:
df.reset_index(inplace=True)
df.drop(['Close', 'Volume'], axis=1, inplace=True)
df.columns = ['date', 'open', 'high', 'low', 'close']

In [14]:
renko = Renko(df)

In [16]:
renko.brick_size = 5
renko_df = renko.get_ohlc_data()

In [18]:
renko_df['uptrend_l1'] = renko_df['uptrend'].shift(periods=1, fill_value=np.nan)

In [19]:
renko_df.head()

,date,open,high,low,close,uptrend,uptrend_l1
0,2010-04-26,15.0,20.0,15.0,20.0,True,NaN
1,2012-02-08,20.0,25.0,20.0,25.0,True,True
2,2013-05-20,25.0,30.0,25.0,30.0,True,True
3,2014-03-20,30.0,35.0,30.0,35.0,True,True
4,2014-08-19,35.0,40.0,35.0,40.0,True,True


In [23]:
renko_df['bar_t'] = renko_df['uptrend'].replace([True, False], [1, -1])
renko_df['bar_t-1'] = renko_df['bar_t'].shift(periods=1, fill_value=np.nan)

In [27]:
renko_df['bar_t-1'].cumsum()

0      NaN
1      1.0
2      2.0
3      3.0
4      4.0
5      5.0
6      6.0
7      7.0
8      8.0
9      9.0
10    10.0
11    11.0
12    12.0
13    13.0
14    14.0
15    15.0
16    16.0
17    17.0
18    18.0
19    19.0
20    18.0
21    19.0
22    18.0
23    17.0
24    18.0
25    19.0
26    20.0
27    21.0
28    22.0
29    23.0
30    24.0
31    25.0
32    26.0
33    27.0
34    28.0
35    29.0
36    30.0
37    31.0
38    32.0
39    33.0
40    34.0
41    33.0
42    32.0
43    31.0
44    30.0
45    31.0
46    30.0
47    29.0
48    28.0
49    27.0
50    26.0
51    27.0
52    28.0
53    27.0
54    26.0
55    27.0
56    28.0
57    29.0
58    30.0
59    31.0
Name: bar_t-1, dtype: float64

In [28]:
    renko_df["bar_num"] = np.where(renko_df["uptrend"]==True,1,np.where(renko_df["uptrend"]==False,-1,0))
    for i in range(1,len(renko_df["bar_num"])):
        if renko_df["bar_num"][i]>0 and renko_df["bar_num"][i-1]>0:
            renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]
        elif renko_df["bar_num"][i]<0 and renko_df["bar_num"][i-1]<0:
            renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]

<ipython-input-28-a50a23a97637>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]
<ipython-input-28-a50a23a97637>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renko_df["bar_num"][i]+=renko_df["bar_num"][i-1]


In [29]:
renko_df

,date,open,high,low,close,uptrend,uptrend_l1,bar_t,bar_t-1,bar_num
0,2010-04-26,15.0,20.0,15.0,20.0,True,NaN,1,NaN,1
1,2012-02-08,20.0,25.0,20.0,25.0,True,True,1,1.0,2
2,2013-05-20,25.0,30.0,25.0,30.0,True,True,1,1.0,3
3,2014-03-20,30.0,35.0,30.0,35.0,True,True,1,1.0,4
4,2014-08-19,35.0,40.0,35.0,40.0,True,True,1,1.0,5
5,2015-10-23,40.0,45.0,40.0,45.0,True,True,1,1.0,6
6,2015-11-06,45.0,50.0,45.0,50.0,True,True,1,1.0,7
7,2016-10-21,50.0,55.0,50.0,55.0,True,True,1,1.0,8
8,2016-12-19,55.0,60.0,55.0,60.0,True,True,1,1.0,9
9,2017-04-28,60.0,65.0,60.0,65.0,True,True,1,1.0,10


In [31]:
renko_df.size


600

In [32]:
renko_df.shape

(60, 10)

In [33]:
renko_df.ndim

2